# Computer Lab : Nonnenegative Matrix Factorization

### Data Base

#### Question 1.1


In [16]:
# Imports needed
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse.linalg import svds
import scipy as sp
import time

In [17]:
# Search in all directories
def build_M_matrix(stop):
    
    M = []
    
    i = 0
    for element in os.listdir('att_faces'):
        if element != 'README':
            for image in os.listdir('att_faces/' + element):
                image_name = 'att_faces/' + element + '/' + image
                M.append(np.ravel(plt.imread(image_name)))
                i += 1
                if stop and i==10:
                    return M

    return M

In [18]:
M = build_M_matrix(False)
M = np.array(M)
M = M.astype('f')

In [19]:
print('Number of images in the database = ' + str(len(M)))
print('Number of pixels in a image = ' + str(M[0].size))
#We suppose here that all images have the same number of pixels 

Number of images in the database = 400
Number of pixels in a image = 10304


### Presentation of the model

#### Question 2.1

We know that if a function $f:\mathbb{R}^{n}\longrightarrow\mathbb{R}$ is convex, then $f:\mathbb{R}^{2}\longrightarrow f(0,...,0,x,0,...,0,y,0,...0)$ is also convex.


Let's prove that the function isn't convex by contraposition.

Let $f:(W_{1,1},...,W_{1,k},...,W_{n,k},H_{1,1},...,H_{1,p},...,H_{k,p})\longrightarrow \dfrac{1}{2np}\sum_{i=1}^{n}\sum_{l=1}^{p}(M_{i,l}-\sum_{j=1}^{k}W_{i,j}H_{j,l})^2$

Let us show that  $f:(x, y)\longrightarrow (1-xy)^2$ is not convex.
If we take for instance the three couples $(-1,-1)$ and $(1,1)$ an $(0,0)$.
We have $f(-1,-1) = 0$ , $f(1,1) = 0$ , $f(0,0) = 1$  
With $t = \dfrac{1}{2}$, we have $f(0,0) > \dfrac{1}{2}f(1,1) + \dfrac{1}{2}f(-1,-1)$  
That shows that $f:(x, y)\longrightarrow (1-xy)^2$ is not convex.

If we take W and H non negative matrices with only one coefficent x or y, we get the previous function we just studied.
That shows that $f:(W_{1,1},...,W_{1,k},...,W_{n,k},H_{1,1},...,H_{1,p},...,H_{k,p})\longrightarrow \dfrac{1}{2np}\sum_{i=1}^{n}\sum_{l=1}^{p}(M_{i,l}-\sum_{j=1}^{k}W_{i,j}H_{j,l})^2$ is not convex.


$f:(W_{1,1},...,W_{1,k},...,W_{n,k},H_{1,1},...,H_{1,p},...,H_{k,p})\longrightarrow \dfrac{1}{2np}\sum_{i=1}^{n}\sum_{l=1}^{p}(M_{i,l}-\sum_{j=1}^{k}W_{i,j}H_{j,l})^2$

We know by definition :

$f(W+w, H+h) = f(W,H) + <\nabla_{W}f(W, H), W> + <\nabla_{H}f(W, H), h> + o(\|wh\|)$

Let's calculate it in another way.

$f(W+w, H+h) = \dfrac{1}{2np}\|M-(W+w)(H+h)\|^{2}$

We develop the scalar product to have :

$f(W+w, H+h) = \dfrac{1}{2np}(<M-WH, M-WH>-2<M-WH, Wh>-2<M-WH, wH>) + o(\|wh\|)$

$f(W+w, H+h) = \dfrac{1}{2np}(f(W,H)-2<M-WH, Wh>-2<M-WH, wH>) + o(\|wh\|)$

We multiply by the adjoint

$f(W+w, H+h) = \dfrac{1}{2np}(f(W,H)-2<W^{T}(M-WH), h>-2<(M-WH)H^{T}, w>) + o(\|wh\|)$

We can identify the gradient :

$\nabla_{H}f(W, H) = \dfrac{-1}{np}W^{T}(M-WH)$

$\nabla_{W}f(W, H) = \dfrac{-1}{np}(M-WH)H^{T}$

Proving that the function is Lipschitz continuous is equivalent to saying that the function is bounded by a linear function. The degree of both gradient is 3. It can't be bounded by a linear function, so the function isn't Lipschitz continuous.

### Find W when H0 is fixed

#### Question 3.1


We use the singular decomposition value. The Matrix $S$ is diagonal and contains the square root of all positive eigenvalues of $S$. It seems relevent to take $\sqrt{S}$

We want a nonnenegative factorization of $M$ as $M \approx WH$. Starting with the SVD is a good start as it gives a good approximation of the factorization. Besides, python algorithm for SVD is fast.

We have $M \approx W_{0}SH_{0}$

$M \approx (W_{0}\sqrt{S})(\sqrt{S}H_{0})$

As we want a factorization of M as the product of two matrices, we can write :

$M \approx (W_{0}S\sqrt{H_{0}})(\sqrt{H_{0}})$ or $ M \approx (\sqrt{W_{0}})(\sqrt{W_{0}}SH_{0})$ but we have to be sure that we have the right to take the quare root of $W_{0}$ and $H_{0}$

#### Question 3.2

To show that g is convex, we use the same method (question 2.1):

Here we take the function : $f:x\longrightarrow(1-\alpha^{2}) \forall \alpha \in \mathbb{R}$. We see that this function is convex. As $g(W) = \dfrac{1}{2np}\|M-WH_{0}\|_{F}^{2}$ we can directly say that g is convex.



$\nabla g = \nabla _{H} f = \dfrac{-1}{np}(M-WH)H^{T}$

#### Question 3.3

In [33]:
def vectorize(W, H):
    return np.concatenate((W.ravel(), H.ravel()))

def unvectorize_M(W_H, M):
    # number of elements in W_H is (n+p)*k where M is of size n x m
    # W has the nk first elements
    # H has the kp last elements
    n, p = M.shape
   
    k = W_H.shape[0] // (n + p)
    W = W_H[:n * k].reshape((n, k))
    H = W_H[n * k:].reshape((k, p))
    return W, H

In [ ]:
W0, S, H0 = svds(M, 3)
W0 = np.maximum(0, W0 * np.sqrt(S))
n, k = W0.shape
H0 = np.maximum(0,(H0.T * np.sqrt(S)).T)
p = H0.shape[1]
vector0 = vectorize(W0, H0)

print("n = " + str(n) + "\nk = " + str(k) + "\np = " + str(p))

In [54]:
def g(W_H):
    W, H = unvectorize_M(W_H, M)
    return np.linalg.norm(M - np.dot(W, H))/(2*M.size)
print("g(W0)  = " + str(g(vector0)))

g(W0)  = 0.008909251527016207


In [55]:
H0

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [2.0830002 , 2.0739896 , 2.0807538 , ..., 0.10482615, 0.09662902,
        0.32288226],
       [3.522834  , 3.5211167 , 3.5350463 , ..., 3.045007  , 3.0107825 ,
        2.981665  ]], dtype=float32)

In [56]:
def gradient_g_b(W_H):
    W, H = unvectorize_M(W_H, M)
    gradient_W = -(1/M.size)*np.dot(M - np.dot(W, H), H.T)
    return gradient_W


In [38]:
#sp.optimize.check_grad(g_function, gradient_g, vector0)

    

#### Question 3.4

Let $\gamma >0$. Let's prove that $prox_{\gamma \tau \mathbb {R}_{+}}$ is the projection onto ${\mathbb {R_{+}}}$

$\forall x \in \mathbb {R}$ we can say that $x = x_{+} + x_{-}$ with $x_{+} \in \mathbb {R}_{+}, x_{-} \in \mathbb {R}_{-}$. If $x>0$, then $x_{-}=0$. If $x<0$, then $x_{+}=0$


$prox_{\gamma \tau \mathbb {R}_{+}} = argmin \tau (y) + \dfrac{1}{2\gamma} \|y-x\|^{2}$

$prox_{\gamma \tau \mathbb {R}_{+}} = argmin \tau (y) + \dfrac{1}{2\gamma} \|y - x_{+} + x_{-}\|^{2}$

We want to minimize $\|y - x_{+} + x_{-}\|^{2}$. If $x_{+} \geq x_{-}$ then $x \in \mathbb {R}_{+}$ and $y = x_{+} + x_{-}=x$

If $x_{+} \leq x_{-}$ then $x \in \mathbb {R}_{-*}$ and $y = x_{+} = 0$

Finally we can say that $prox_{\gamma \tau \mathbb {R}_{+}}$ is the projection onto ${\mathbb {R_{+}}}$


#### Question 3.5

In [39]:
gamma = np.linalg.norm(np.dot(H0.T, H0))
print(gamma)

246380.5


In [57]:

def projected_gradient_method(val_g, grad_g, W0_H0, gamma, N):
    W0, H0 = unvectorize_M(W0_H0, M)
    count = 0
    xk = W0
    while count < N:
        xk = np.maximum(0, xk - gamma*grad_g(vectorize(xk, H0)))
        count += 1
    return xk, val_g(vectorize(xk, H0))

#### Question 3.6

In [58]:
start = time.time()

Wmin_1, g_Wmin_1 = projected_gradient_method(g, gradient_g_b, vector0, 2, 100)

end = time.time()
exe_time_1 = end - start
print(Wmin_1, g_Wmin_1)

[[12.636662    0.18920523 26.735384  ]
 [ 1.7368478   0.         28.348639  ]
 [ 1.2631546   0.         30.60054   ]
 ...
 [ 0.          9.769409   23.40776   ]
 [ 0.         11.4472885  23.750692  ]
 [ 0.         14.107467   22.776457  ]] 0.008761269705636161


## Algorithmic refinement for the problem with H0 fixed

#### Question 4.1


In order to get rid of the Lipschitz constant value, we perform a line search. We want to find a new step at each iteration. This step is given by $\gamma _{k} = arg min f(x_{k}-\gamma \nabla f(x_{k}))$
It is difficult to get a argmin over R+. We just create a list with enought possible values for gamma and search through the list the argmin

In [59]:
def g_line_search_gamma(gamma, W_H):
    W, H = unvectorize_M(W_H, M)
    return g(vectorize(W - gamma*gradient_g_b(W_H), H))


#We built a list with gamma values to search for the minimum. We try to select as many values as possible to have a great step
def line_search(function, gradient_function, W0_H0, N):
    W0, H0 = unvectorize_M(W0_H0, M)
    count = 0
    xk = W0
    gamma_list = np.linspace(0.000001, 10, 10)
    while count < N:
        xk_vector = vectorize(xk, H0)
        gggggggggggggg = np.vectorize(g_line_search_gamma, excluded=['W_H'])
        gamma_list_g = gggggggggggggg(gamma_list, W_H=xk_vector)
        index = np.argmin(gamma_list_g)
        gamma = gamma_list[index] #Value of the step
        xk = np.maximum(0, xk - gamma*gradient_function(xk_vector))
        count += 1
    return xk, g(vectorize(xk, H0))




In [44]:
start = time.time()

Wmin_2, g_Wmin_2 = line_search(g, gradient_g_b, vector0, 100)

end = time.time()
exe_time_2 = end - start
print(Wmin_2, g_Wmin_2)

[[16.160194   0.9432766 26.250122 ]
 [ 0.         0.        28.538973 ]
 [ 4.512787   0.        30.253246 ]
 ...
 [ 0.        13.786078  22.822712 ]
 [ 0.        15.991316  23.088829 ]
 [ 0.        19.37718   22.008894 ]] 0.008692754899492915


#### Question 4.2

In [45]:
print("Execution time : \nProjected gradient method = " + str(exe_time_1) + "\nLine search method : " + str(exe_time_2))

print("W value : \nProjected gradient method = " + str(Wmin_1) + "\nLine search method : " + str(Wmin_2))

print("g(W) value : \nProjected gradient method = " + str(g_Wmin_1) + "\nLine search method : " + str(g_Wmin_2))



Execution time : 
Projected gradient method = 3.576655626296997
Line search method : 81.55647802352905
W value : 
Projected gradient method = [[12.636662    0.18920523 26.735384  ]
 [ 1.7368478   0.         28.348639  ]
 [ 1.2631546   0.         30.60054   ]
 ...
 [ 0.          9.769409   23.40776   ]
 [ 0.         11.4472885  23.750692  ]
 [ 0.         14.107467   22.776457  ]]
Line search method : [[16.160194   0.9432766 26.250122 ]
 [ 0.         0.        28.538973 ]
 [ 4.512787   0.        30.253246 ]
 ...
 [ 0.        13.786078  22.822712 ]
 [ 0.        15.991316  23.088829 ]
 [ 0.        19.37718   22.008894 ]]
g(W) value : 
Projected gradient method = 0.008761269705636161
Line search method : 0.008692754899492915


The line search is longer because of the gamma search. We are search the argmin on a list with length 10000.

Finally, the value found with the projected gradient method is more precise, with 100 iterations (it doesn't stop before in both case, I've checked it) it is less than line search value. this is because the step is not optimized. With a Lipschitz function, we have the best step we can find. Here we are calculating gamma by hand. The final value isn't very suprising.

## Resolution of the full problem

#### Question 5.1

In [60]:
def f(W_H):
    #print("W_H dans f = " + str(W_H))
    W, H = unvectorize_M(W_H, M)
    return np.linalg.norm(M - np.dot(W, H))/(2*M.size)

def grad_f(W_H):
    W, H = unvectorize_M(W_H, M)
    gradient_W = -(1/M.size)*np.dot(M - np.dot(W, H), H.T)
    gradient_H = -(1/M.size)*np.dot(W.T, M - np.dot(W, H))
    return vectorize(gradient_W, gradient_H)
    

def g_line_search_gammaf(gamma, W_H):
    #print("W_H dans f = " + str(W_H))
    return f(W_H - gamma*grad_f(W_H))


def line_search_final(function, gradient_function, W0, H0, N):
    count = 0
    xk = vectorize(W0, H0)
    #print("W_H dans line search avant boucle")
    gamma_list = np.linspace(0.000001, 100, 10)
    while count < N:
        gamma_list_g = np.vectorize(g_line_search_gammaf, excluded=['W_H'])
        #print("W_H dans line search boucle = " + str(xk))
        gamma_list_gg = gamma_list_g(gamma_list, W_H=xk)
        index = np.argmin(gamma_list_gg)
        gamma = gamma_list[index] #Value of the step
        xk = np.maximum(0, xk - gamma*gradient_function(xk))
        count += 1
    return xk, function(xk) 

In [61]:
start = time.time()

W_Hfinal_projected, ffinal_projected = line_search_final(f, grad_f, W0, H0, 10)

end = time.time()
exe_time_projected = end-start

print("W_H minimum = " + str(W_Hfinal_projected))
print("Minimum of the function (1) = " + str(ffinal_projected))

W_H minimum = [14.212811   2.0744386 26.849216  ...  3.0367348  2.9991405  2.9226599]
Minimum of the function (1) = 0.008519378093458851


#### Question 5.2

We want to show that the value of the objective is decreasing at each iteration. This is equivalent to show that $\forall t \in \mathbb {R}, \|M-W_{t}H_{t}\|^{2} \leq \|M-W_{t-1}H_{t-1}\|^{2}$

$arg min \|M-WH_{t-1}\|^{2} \leq \|M-W_{t-1}H_{t-1}\|^{2}$

$\|M-W_{t}H_{t-1}\|^{2} \leq \|M-W_{t-1}H_{t-1}\|^{2}$

$ arg min \|M-W_{t}H\|^{2} \leq \|M-W_{t}H_{t-1}\|^{2}$

$\|M-W_{t}H_{t}\|^{2} \leq \|M-W_{t}H_{t-1}\|^{2}$

Finally: $\forall t \in \mathbb {R}, \|M-W_{t}H_{t}\|^{2} \leq \|M-W_{t-1}H_{t-1}\|^{2}$

Let $h : t \longrightarrow \dfrac{1}{2np}\|M-W_{t}H_{t}\|^{2}$ $h$ is decreasing and bounded so h converges.

#### Question 5.3

In [62]:
def gradient_g_b_H(W_H):
    W, H = unvectorize_M(W_H, M)
    gradient_H = -(1/M.size)*np.dot(W.T, M - np.dot(W, H))
    return gradient_H

def projected_gradient_method_H(val_g, grad_g, W0_H0, gamma, N):
    W0, H0 = unvectorize_M(W0_H0, M)
    count = 0
    xk = H0
    while count < N:
        xk = np.maximum(0, xk - gamma*grad_g(vectorize(W0, xk)))
        count += 1
    return xk, val_g(vectorize(W0, xk))





def alternate_minimization(H0_W0, N):
    Wt, Ht = unvectorize_M(H0_W0, M)
    
    for n in range(N):
        Wt = projected_gradient_method(g, gradient_g_b, vector0, 2, 100)[0]
        Wt_Ht = vectorize(Wt, Ht)
        Ht = projected_gradient_method_H(g, gradient_g_b_H, Wt_Ht, 2, 100)[0]
    
    return vectorize(Wt, Ht), g(vectorize(Wt, Ht))
        
        

In [ ]:
start = time.time()

W_Hfinal_alternate, ffinal_alternate = alternate_minimization(vector0, 100)

end = time.time()

exe_time_alternate = end-start

#### Question 5.4

In [111]:
# Comparison 

print("Projected Gradient : \nW_Hfinal = " + str(W_Hfinal_projected) + 
      "\Function value = " + str(ffinal_projected) + 
      "\nExecution time = " + str(exe_time_projected))

print("Alternate Minimizations : \nW_Hfinal = " + str(W_Hfinal_alternate) + 
      "\Function value = " + str(ffinal_alternate) + 
      "\nExecution time = " + str(exe_time_alternate)))


Projected Gradient : 
W_Hfinal = [ 11.26682281   2.32865405  25.95561028 ...,   3.03673744   2.99914265
   2.92266226]\Function value = 0.00851928426612
Execution time = 4.971767902374268


#### Question 5.5

We could have stop thanks to a threshold : we know that for $W_H* = arg min f(W, H)$, then $\nabla f(W_H*) = 0$. So if we calculate the gradient for each $x_{k}$ and say that : $\|\nabla f(x_{k})\| < \epsilon$ with $\epsilon$ small enough, it is a criterion to stop the algorithm.